# 2021.02.17

**-Use VGG16, but also gonna try with ResNets**   
**-Normalization added!**   
**-<span style = "background-color : pink">The last test for train data</span>**

..Grayscale worked well as muas as color did!

<h3><span style = "background-color : red">Color Test</span></h3><br>
    - <strong>Using VGG16</strong><br>
    - <strong>Using color image data</strong>

<h3>Alert!</h3>
- <strong><u>If lr (Learning Rate) is small and decay is big</u>,<br>1) val_loss easily converges to 0 and<br>2) learning ends so fast that ACC never gets better (0.5, 1, asf.. - extreme value)<strong>

<h4> <span style = "background-color : skyblue">At Desk </span></h4>

In [1]:
import warnings
warnings.filterwarnings(action = "ignore")

import pickle
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

c_F = pickle.load(open("imagedata/Desk/color_train2_F.pickle", "rb"))
c_T = pickle.load(open("imagedata/Desk/color_train2_T.pickle", "rb"))

In [2]:
x = np.concatenate( (c_F, c_T), axis = 0 )


y = [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))]

X = []

for rows in x:
    tmp = cv2.resize(rows, (224, 224))
    X.append(tmp)
    
X = np.array(X)
X = X.reshape(-1, 224, 224, 3)

In [3]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

In [5]:
## Normalization
X = X / 255.0

y = np.array(y)

nums = [i for i in range(8000)]

import random

random.seed(4)
random.shuffle(nums)

X = X[nums]
y = y[nums]

In [6]:
INPUT_SHAPE = (224, 224, 3)

model = Sequential()

# 1st Convolutional Layer
model.add( Conv2D(input_shape= INPUT_SHAPE, filters=64, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 2nd Convolutional Layer
model.add( Conv2D(filters=64, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 3rd Convolutional Layer
model.add( Conv2D(filters=128, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 4th Convolutional Layer
model.add( Conv2D(filters=128, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 5th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 6th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 7th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 8th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 9th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 10th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 11th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 12th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 13th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))



########################## Passing it to a Fully Connected layer ##########################
model.add( Flatten() )

K = INPUT_SHAPE[0] * INPUT_SHAPE[1] * INPUT_SHAPE[2]

# 1st Fully Connected Layer
model.add( Dense(4096, input_shape=( K, )) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 2nd Fully Connected Layer
model.add( Dense(4096) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# Output Layer
model.add( Dense(2) )
model.add( Activation('softmax') )
          
opt = tf.keras.optimizers.Adam(lr = 0.001)

model.compile(loss = "sparse_categorical_crossentropy", # or binary_crossentropy
             optimizer = opt,
             metrics = ['accuracy'])

model.summary()

model.fit(X, y, batch_size = 8, epochs = 10, validation_split = 0.1)  # batch_size

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
____

In [8]:
# model.save("0217_alexnet.model")

## *Test Data Pre-processing*

In [7]:
test_F = pickle.load(open("imagedata/Desk/color_test2_F.pickle", "rb"))
test_T = pickle.load(open("imagedata/Desk/color_test2_T.pickle", "rb"))

test_x = np.concatenate( (test_F, test_T), axis = 0 )


test_y = [1 for i in range(len(test_F))] + [0 for i in range(len(test_T))]

test_X = []

for rows in test_x:
    tmp = cv2.resize(rows, (224, 224))
    test_X.append(tmp)
    
test_X = np.array(test_X)
test_X = test_X.reshape(-1, 224, 224, 3)

test_X = test_X / 255.0

test_y = np.array(test_y)

nums = [i for i in range(2000)]

import random

random.seed(6)
random.shuffle(nums)

test_X = test_X[nums]
test_y = test_y[nums]

### Evaluate!

In [9]:
results = model.evaluate(test_X, test_y, batch_size=8)

2000/2000 [==============================] - 4s 2ms/sample - loss: 7.1526e-09 - acc: 1.0000 1s - loss: 7.748 - ETA: 0s - loss: 7.4876e-09 


In [10]:
print(results)

[7.152555408396211e-09, 1.0]


**Did I code properly?**

In [26]:
k = 719

predictions = model.predict([test_X[k].reshape(1, 224, 224, 3)])
print(np.argmax(predictions[0]))

cv2.imshow("title", test_X[k])
cv2.waitKey(0)

1


-1

<h3><span style = "background-color : gray">Gray (<span style = "background-color : red">from Color</span>) Test</span></h3><br>
    - <strong>Using VGG16</strong><br>
    - <strong>Using gray-scale image data</strong>

In [13]:
import warnings
warnings.filterwarnings(action = "ignore")

import pickle
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization



c_F = pickle.load(open("color_train_F.pickle", "rb"))
c_T = pickle.load(open("color_train_T.pickle", "rb"))

### Change color into gray & resize from color images

In [14]:
F = []
for i in range(c_F.shape[0]):
    F.append(cv2.resize( cv2.cvtColor(c_F[i], cv2.COLOR_BGR2GRAY), (224, 224) ))

T = []
for j in range(c_T.shape[0]):
    T.append(cv2.resize( cv2.cvtColor(c_T[i], cv2.COLOR_BGR2GRAY), (224, 224) ))
    
F, T = np.array(F), np.array(T)

In [15]:
X = np.concatenate( (F, T), axis = 0 )
X = X.reshape(-1, 224, 224, 1)

y = [1 for i in range(len(F))] + [0 for i in range(len(T))]
y = np.array(y)

nums = [i for i in range(c_F.shape[0] + c_T.shape[0])]

import random

random.seed(4)
random.shuffle(nums)

X = X[nums]
y = y[nums]

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

INPUT_SHAPE = (224, 224, 1)

model = Sequential()

# 1st Convolutional Layer
model.add( Conv2D(input_shape= INPUT_SHAPE, filters=64, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 2nd Convolutional Layer
model.add( Conv2D(filters=64, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 3rd Convolutional Layer
model.add( Conv2D(filters=128, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 4th Convolutional Layer
model.add( Conv2D(filters=128, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 5th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 6th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 7th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 8th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 9th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 10th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(2,2), strides=(2,2)) )

# 11th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 12th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# 13th Convolutional Layer
model.add( Conv2D(filters=512, kernel_size=(3,3), strides = (1, 1), padding="same") )
model.add( Activation('relu') )

# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))



########################## Passing it to a Fully Connected layer ##########################
model.add( Flatten() )

K = INPUT_SHAPE[0] * INPUT_SHAPE[1] * INPUT_SHAPE[2]

# 1st Fully Connected Layer
model.add( Dense(4096, input_shape=( K, )) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 2nd Fully Connected Layer
model.add( Dense(4096) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# Output Layer
model.add( Dense(2) )
model.add( Activation('softmax') )
          
opt = tf.keras.optimizers.Adam(lr = 0.001)

model.compile(loss = "sparse_categorical_crossentropy", # or binary_crossentropy
             optimizer = opt,
             metrics = ['accuracy'])

model.summary()

model.fit(X, y, batch_size = 8, epochs = 10, validation_split = 0.1)  # batch_size

## Epoch doesn't work well?
Refer to this; https://stackoverflow.com/questions/37213388/keras-accuracy-does-not-change